# Minian reduced

## Load packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%%capture  ## "%%capture var" redirects stdout and stderr to the variable var that can be used latter on. If no var is provided, it will just suppress the output
import itertools as itt
import os
import sys

import holoviews as hv
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from IPython.display import display
from ipyfilechooser import FileChooser


In [3]:
cd "C:/Users/Manip2/SCRIPTS/Code python audrey/code python aurelie/interfaceJupyter/minian"

C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian


In [4]:
#%%capture
minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))

sys.path.append(minian_path)
from minian.cnmf import (
    compute_AtC,
    compute_trace,
    get_noise_fft,
    smooth_sig,
    unit_merge,
    update_spatial,
    update_temporal,
    update_background,
)
from minian.initialization import (
    gmm_refine,
    initA,
    initC,
    intensity_refine,
    ks_refine,
    pnr_refine,
    seeds_init,
    seeds_merge,
)
from minian.motion_correction import apply_transform, estimate_motion
from minian.preprocessing import denoise, remove_background
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)
from minian.visualization import (
    CNMFViewer,
    VArrayViewer,
    generate_videos,
    visualize_gmm_fit,
    visualize_motion,
    visualize_preprocess,
    visualize_seeds,
    visualize_spatial_update,
    visualize_temporal_update,
    write_video,
)

The folder used for minian procedures is : C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian


## Configuration

### Select folder

In [5]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/RAW"
try:
    %store -r dpath
    #dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/RAW"
except:
    print("data not in strore")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/RAW"

# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)


FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\RAW\Baseline_recording\BlackLines\Baselin…

In [8]:
minian_ds_path = os.path.join(dpath, "minianAB")
intpath = os.path.join(dpath, "minian_intermediateAB")
minian_ds_path

'\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\minianAB'

### Initial parameters

In [9]:
subset = dict(frame=slice(0, None))
subset_mc = None
interactive = True
output_size = 100
n_workers = int(os.getenv("MINIAN_NWORKERS", 4))
param_save_minian = {
    "dpath": minian_ds_path,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True,
}

# Pre-processing Parameters#
param_load_videos = {
    "pattern": "[0-9]+\.avi$",
    "dtype": np.uint8,
    "downsample": dict(frame=1, height=1, width=1),
    "downsample_strategy": "subset",
}
param_denoise = {"method": "median", "ksize": 5}
param_background_removal = {"method": "tophat", "wnd": 15}

# Motion Correction Parameters#
subset_mc = None
param_estimate_motion = {"dim": "frame"}

# Initialization Parameters#
param_seeds_init = {
    "wnd_size": 100,
    "method": "rolling",
    "stp_size": 50,
    "max_wnd": 20,#generally 10 updated here to 20 to account for L1 wide dendritic trees
    "diff_thres": 3,
}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06} 
param_init_merge = {"thres_corr": 0.8}

# CNMF Parameters# 0.025 for threecolordots
param_get_noise = {"noise_range": (0.06, 0.5)}
param_first_spatial = {
    "dl_wnd": 15,
    "sparse_penal": 0.012,
    "size_thres": (25, None),
}
param_first_temporal = {
    "noise_freq": 0.06,
    "sparse_penal": 1,
    "p": 1,
    "add_lag": 20,
    "jac_thres": 0.2,
}
param_first_merge = {"thres_corr": 0.8}
param_second_spatial = {
    "dl_wnd": 10,
    "sparse_penal": 0.005,
    "size_thres": (25, None),
}
param_second_temporal = {
    "noise_freq": 0.06,
    "sparse_penal": 1,
    "p": 1,
    "add_lag": 20,
    "jac_thres": 0.4,
}

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MINIAN_INTERMEDIATE"] = intpath

## start cluster

In [10]:
hv.notebook_extension("style=dict(cmap=Viridis256),")
hv.notebook_extension("bokeh", width=70)

cluster = LocalCluster(
        n_workers=n_workers,
        memory_limit="8GB", #4
        resources={"MEM": 1},
        threads_per_worker=2,
        dashboard_address=":8780" #port 8787 already used by jupyter
    )

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [11]:
annt_plugin = TaskAnnotation()
cluster.scheduler.add_plugin(annt_plugin)
client = Client(cluster)
print(cluster)
print(client)

LocalCluster(e037b347, 'tcp://127.0.0.1:64319', workers=4, threads=8, memory=29.80 GiB)
<Client: 'tcp://127.0.0.1:64319' processes=4 threads=8, memory=29.80 GiB>


## Pre-processing

In [13]:
%%time
#%%capture

varr = load_videos(dpath, **param_load_videos)
chk, _ = get_optimal_chk(varr, dtype=np.float64)

varr = save_minian(
    varr.chunk({"frame": chk["frame"], "height": -1, "width": -1}).rename("varr"),
    intpath,
    overwrite=True,
)

['\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\0.avi', '\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\1.avi', '\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\2.avi', '\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\3.avi', '\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\4.avi', '\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\5.avi']
0
1
2
3
4
5
loading 6

In [14]:
#possibility to crop data
varr_ref = varr.sel(height=slice(0, 600), width=slice(0, 600))

## Clean up

### Glow removal

In [15]:
%%time
varr_min = varr_ref.min("frame").compute()
varr_ref = varr_ref - varr_min

CPU times: total: 31.2 ms
Wall time: 5.53 s


### Denoise
Make sure to update the [denoise parameters](#Initial-parameters) based on what you see before proceeding

In [16]:
%%time
varr_ref = denoise(varr_ref, **param_denoise)

CPU times: total: 0 ns
Wall time: 6.62 ms


### Background removal
Make sure to update the [background removal parameters](#Initial-parameters) based on what you see before proceeding

In [17]:
%%time
varr_ref = remove_background(varr_ref, **param_background_removal)

CPU times: total: 0 ns
Wall time: 6.09 ms


### Save results

In [18]:
%%time
varr_ref = save_minian(varr_ref.rename("varr_ref"), dpath=intpath, overwrite=True)

CPU times: total: 219 ms
Wall time: 15.3 s


## Motion correction

### Estimation motion

In [19]:
%%time
motion = estimate_motion(varr_ref.sel(subset_mc), **param_estimate_motion)

CPU times: total: 0 ns
Wall time: 101 ms


### Save motion

In [20]:
%%time
motion = save_minian(
    motion.rename("motion").chunk({"frame": chk["frame"]}), **param_save_minian
)

CPU times: total: 1.97 s
Wall time: 2min 36s


### Visualization of motion

### Apply transform

In [21]:
Y = apply_transform(varr_ref, motion, fill=0)

### Save result

In [22]:
%%time
Y_fm_chk = save_minian(Y.astype(float).rename("Y_fm_chk"), intpath, overwrite=True)
Y_hw_chk = save_minian(
    Y_fm_chk.rename("Y_hw_chk"),
    intpath,
    overwrite=True,
    chunks={"frame": -1, "height": chk["height"], "width": chk["width"]},
)

CPU times: total: 2.14 s
Wall time: 4min 37s


### Visualization of motion-correction

### Generate video for motion-correction

In [23]:
%%time
vid_arr = xr.concat([varr_ref, Y_fm_chk], "width").chunk({"width": -1})
write_video(Y_fm_chk, "minian_mc.mp4", dpath)

CPU times: total: 1.62 s
Wall time: 51.3 s


'\\\\10.69.168.1\\crnldata\\waking\\audrey_hay\\L1imaging\\RAW\\Baseline_recording\\BlackLines\\Baseline_recording\\2022_08_10\\12_50_58\\My_V4_Miniscope\\minian_mc.mp4'

## Initialisation 

### Compute maximal projection

In [24]:
max_proj = save_minian(
    Y_fm_chk.max("frame").rename("max_proj"), **param_save_minian
).compute()

### Generating seeds

In [25]:
%%time
seeds = seeds_init(Y_fm_chk, **param_seeds_init)

constructing chunks
computing max projections
calculating local maximum
CPU times: total: 938 ms
Wall time: 1min 9s


### Noise refined 
here there is possibility to visualise to refine param_pnr_refine but noise_freq = 0.06 is generally fine

In [26]:
%%time
seeds, pnr, gmm = pnr_refine(Y_hw_chk, seeds, **param_pnr_refine)

selecting seeds
computing peak-noise ratio
CPU times: total: 172 ms
Wall time: 14 s


### Refine using KS test to look at bimodal distribution

In [27]:
%%time
seeds = ks_refine(Y_hw_chk, seeds, **param_ks_refine)

selecting seeds
performing KS test
CPU times: total: 266 ms
Wall time: 7.94 s


### Merge seeds

In [28]:
%%time
seeds_final = seeds[seeds["mask_ks"] & seeds["mask_pnr"]].reset_index(drop=True)
seeds_final = seeds_merge(Y_hw_chk, max_proj, seeds_final, **param_seeds_merge)
print("{} units found".format(seeds_final["mask_mrg"].count()))

computing distance
computing correlations
pixel recompute ratio: 0.9636752136752137
computing correlations
merging seeds
468 units found
CPU times: total: 78.1 ms
Wall time: 6.91 s


### Initialise spatial matrix

In [29]:
%%time
A_init = initA(Y_hw_chk, seeds_final[seeds_final["mask_mrg"]], **param_initialize)
A_init = save_minian(A_init.rename("A_init"), intpath, overwrite=True)
display(A_init)

optimizing computation graph
pixel recompute ratio: 1.0078786851914605
computing correlations
building spatial matrix


<xarray.DataArray 'A_init' (unit_id: 153, height: 600, width: 600)>
dask.array<from-zarr, shape=(153, 600, 600), dtype=float64, chunksize=(1, 600, 600), chunktype=numpy.ndarray>
Coordinates:
  * height   (height) int32 0 1 2 3 4 5 6 7 ... 592 593 594 595 596 597 598 599
  * unit_id  (unit_id) int32 0 1 6 7 9 10 13 15 ... 461 462 463 464 465 466 467
  * width    (width) int32 0 1 2 3 4 5 6 7 8 ... 592 593 594 595 596 597 598 599

CPU times: total: 2.61 s
Wall time: 21.2 s


### Initialise temporal matrix

In [30]:
%%time
C_init = initC(Y_fm_chk, A_init)
C_init = save_minian(
    C_init.rename("C_init"), intpath, overwrite=True, chunks={"unit_id": 1, "frame": -1}
)

CPU times: total: 1.5 s
Wall time: 1min 44s


### Merge unit

In [31]:
%%time
A_merged, C_merged = unit_merge(A_init, C_init, **param_init_merge)
A_merged = save_minian(A_merged.rename("A"), intpath, overwrite=True)
C_merged = save_minian(C_merged.rename("C"), intpath, overwrite=True)
C_chk_merged = save_minian(
    C_merged.rename("C_chk"),
    intpath,
    overwrite=True,
    chunks={"unit_id": -1, "frame": chk["frame"]},
)

computing spatial overlap
computing temporal correlation
pixel recompute ratio: 0.11764705882352941
computing correlations
labeling units to be merged
merging units
CPU times: total: 719 ms
Wall time: 14.1 s


### Initialise background terms

In [32]:
%%time
b_init, f_init = update_background(Y_fm_chk, A_merged, C_chk_merged)
f_init = save_minian(f_init.rename("f"), intpath, overwrite=True)
b_init = save_minian(b_init.rename("b"), intpath, overwrite=True)

CPU times: total: 281 ms
Wall time: 49.3 s


## CNMF

### Estimate spatial noise

In [33]:
%%time
sn_spatial = get_noise_fft(Y_hw_chk, **param_get_noise)
sn_spatial = save_minian(sn_spatial.rename("sn_spatial"), intpath, overwrite=True)

CPU times: total: 281 ms
Wall time: 34 s


### First spatial update

#### Randomly select a subset of units for exploration

In [36]:
if interactive:
    unitsSub = np.random.choice(A_merged.coords["unit_id"], 10, replace=False)
    unitsSub.sort()
    A_sub = A_merged.sel(unit_id=unitsSub).persist()
    C_sub = C_merged.sel(unit_id=unitsSub).persist()

#### Parameter exploration
Here is the only interactive mandatory step to adjust sparse penalty [generally set between 0.01 and 0.02]
> **WARNING** 
> **Be very careful here!**: The parameter displayed isn't necessarily the one used

In [37]:
%%time
if interactive:
    sprs_ls = [0.005, 0.01, 0.015, 0.02, 0.025]
    A_dict = dict()
    C_dict = dict()
    for cur_sprs in sprs_ls:
        cur_A, cur_mask, cur_norm = update_spatial(
            Y_hw_chk,
            A_sub,
            C_sub,
            sn_spatial,
            in_memory=True,
            dl_wnd=param_first_spatial["dl_wnd"],
            sparse_penal=cur_sprs,
        )
        if cur_A.sizes["unit_id"]:
            A_dict[cur_sprs] = cur_A.compute()
            C_dict[cur_sprs] = C_sub.sel(unit_id=cur_mask).compute()
    hv_res = visualize_spatial_update(A_dict, C_dict, kdims=["sparse penalty"])
    display(hv_res)

estimating penalty parameter
computing subsetting matrix
fitting spatial matrix
0 out of 10 units dropped
estimating penalty parameter
computing subsetting matrix
fitting spatial matrix
0 out of 10 units dropped
estimating penalty parameter
computing subsetting matrix
fitting spatial matrix
0 out of 10 units dropped
estimating penalty parameter
computing subsetting matrix
fitting spatial matrix
0 out of 10 units dropped
estimating penalty parameter
computing subsetting matrix
fitting spatial matrix
0 out of 10 units dropped


c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 27.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 27.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 27.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 27.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.con

BokehModel(combine_events=True, render_bundle={'docs_json': {'2ca5bd74-4531-48b7-98ed-c6bfa5e28e55': {'version…

CPU times: total: 2.86 s
Wall time: 41.8 s


#### Spatial updates

In [63]:
%%time
A_firstS, mask_firstS, norm_fac_firstS = update_spatial(
    Y_hw_chk, A_merged, C_merged, sn_spatial, **param_first_spatial
)
C_firstS = save_minian(
    (C_merged.sel(unit_id=mask_firstS) * norm_fac_firstS).rename("C_new"), intpath, overwrite=True
)
C_chk_firstS = save_minian(
    (C_chk_merged.sel(unit_id=mask_firstS) * norm_fac_firstS).rename("C_chk_new"), intpath, overwrite=True
)

estimating penalty parameter
computing subsetting matrix
fitting spatial matrix


c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 13.04 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 15.72 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


0 out of 460 units dropped
CPU times: total: 2.34 s
Wall time: 2min 26s


#### Background updates

In [64]:
%%time
b_firstS, f_firstS = update_background(Y_fm_chk, A_firstS, C_chk_firstS)

c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 10.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: total: 8.05 s
Wall time: 3min 11s


#### visualization of spatial footprints

#### visualization of background

#### Save results first spatial update

In [67]:
%%time
A_firstS = save_minian(
    A_firstS.rename("A"),
    intpath,
    overwrite=True,
    chunks={"unit_id": 1, "height": -1, "width": -1},
)
b_firstS = save_minian(b_firstS.rename("b"), intpath, overwrite=True)
f_firstS = save_minian(
    f_firstS.chunk({"frame": chk["frame"]}).rename("f"), intpath, overwrite=True
)
C_firstS = save_minian(C_firstS.rename("C"), intpath, overwrite=True)
C_chk_firstS = save_minian(C_chk_firstS.rename("C_chk"), intpath, overwrite=True)

<xarray.DataArray 'A' (unit_id: 460, height: 600, width: 600)>
dask.array<from-zarr, shape=(460, 600, 600), dtype=float64, chunksize=(1, 600, 600), chunktype=numpy.ndarray>
Coordinates:
  * height   (height) int32 0 1 2 3 4 5 6 7 ... 592 593 594 595 596 597 598 599
  * unit_id  (unit_id) int64 0 1 2 3 4 5 6 7 ... 452 453 454 455 456 457 458 459
  * width    (width) int32 0 1 2 3 4 5 6 7 8 ... 592 593 594 595 596 597 598 599

CPU times: total: 984 ms
Wall time: 5.06 s


### First temporal update

#### Randomly select a subset of units for exploration

In [68]:
if interactive:
    unitsSub = np.random.choice(A_firstS.coords["unit_id"], 10, replace=False)
    unitsSub.sort()
    A_sub = A_firstS.sel(unit_id=unitsSub).persist()
    C_sub = C_firstS.sel(unit_id=unitsSub).persist()

#### Parameter exploration

In [70]:
%%time
%env SPARSE_AUTO_DENSIFY=1

if interactive:
    p_ls = [1]
    sprs_ls = [0.1, 0.5, 1, 2]
    add_ls = [20]
    noise_ls = [0.06]
    YA_dict, C_dict, S_dict, g_dict, sig_dict, A_dict = [dict() for _ in range(6)]
    YrA = (
        compute_trace(Y_fm_chk, A_sub, b_firstS, C_sub, f_firstS)
        .persist()
        .chunk({"unit_id": 1, "frame": -1})
    )
    for cur_p, cur_sprs, cur_add, cur_noise in itt.product(
        p_ls, sprs_ls, add_ls, noise_ls
    ):
        ks = (cur_p, cur_sprs, cur_add, cur_noise)
        print(
            "p:{}, sparse penalty:{}, additional lag:{}, noise frequency:{}".format(
                cur_p, cur_sprs, cur_add, cur_noise
            )
        )
        cur_C, cur_S, cur_b0, cur_c0, cur_g, cur_mask = update_temporal(
            A_sub,
            C_sub,
            YrA=YrA,
            sparse_penal=cur_sprs,
            p=cur_p,
            use_smooth=True,
            add_lag=cur_add,
            noise_freq=cur_noise,
        )
        YA_dict[ks], C_dict[ks], S_dict[ks], g_dict[ks], sig_dict[ks], A_dict[ks] = (
            YrA.compute(),
            cur_C.compute(),
            cur_S.compute(),
            cur_g.compute(),
            (cur_C + cur_b0 + cur_c0).compute(),
            A_sub.compute(),
        )
    hv_res = visualize_temporal_update(
        YA_dict,
        C_dict,
        S_dict,
        g_dict,
        sig_dict,
        A_dict,
        kdims=["p", "sparse penalty", "additional lag", "noise frequency"],
    )
    display(hv_res)

env: SPARSE_AUTO_DENSIFY=1


c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 10
  result = blockwise(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 11.18 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


p:1, sparse penalty:0.1, additional lag:20, noise frequency:0.06
grouping overlaping units
updating temporal components
0 out of 10 units dropped
p:1, sparse penalty:0.5, additional lag:20, noise frequency:0.06
grouping overlaping units
updating temporal components
0 out of 10 units dropped
p:1, sparse penalty:1, additional lag:20, noise frequency:0.06
grouping overlaping units
updating temporal components
0 out of 10 units dropped
p:1, sparse penalty:2, additional lag:20, noise frequency:0.06
grouping overlaping units
updating temporal components
2 out of 10 units dropped


BokehModel(combine_events=True, render_bundle={'docs_json': {'86210798-27b4-44e5-bbe0-32d8649b9651': {'version…

CPU times: total: 2.8 s
Wall time: 14 s


#### Temporal update

In [71]:
%%time
YrA_firstT = save_minian(
    compute_trace(Y_fm_chk, A_firstS, b_firstS, C_chk_firstS, f_firstS).rename("YrA"),
    intpath,
    overwrite=True,
    chunks={"unit_id": 1, "frame": -1},
)

c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 10.41 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 10.41 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 21.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: total: 2.05 s
Wall time: 26.2 s


In [72]:
%%time
C_firstT, S_firstT, b0_firstT, c0_firstT, g_firstT, mask_firstT = update_temporal(
    A_firstS, C_firstS, YrA=YrA_firstT, **param_first_temporal
)

grouping overlaping units
updating temporal components
9 out of 460 units dropped
CPU times: total: 5.92 s
Wall time: 1min 16s


#### Visualization of temporal components

#### Visualization of dropped units

#### Visualization of accepted units

#### Save results

In [76]:
%%time
C_firstT = save_minian(
    C_firstT.rename("C").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
C_chk_firstT = save_minian(
    C_firstT.rename("C_chk"),
    intpath,
    overwrite=True,
    chunks={"unit_id": -1, "frame": chk["frame"]},
)
S_firstT = save_minian(
    S_firstT.rename("S").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
b0_firstT = save_minian(
    b0_firstT.rename("b0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
c0_firstT = save_minian(
    c0_firstT.rename("c0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
A_firstT = A_firstS.sel(unit_id=C_firstT.coords["unit_id"].values)

CPU times: total: 1.66 s
Wall time: 6.79 s


### Merge units

In [77]:
%%time
A_mrg, C_mrg, [sig_mrg] = unit_merge(A_firstT, C_firstT, [C_firstT + b0_firstT + c0_firstT], **param_first_merge)

computing spatial overlap
computing temporal correlation
pixel recompute ratio: 0.9889135254988913
computing correlations
labeling units to be merged
merging units
CPU times: total: 1.31 s
Wall time: 4.7 s


#### Save merged units

In [78]:
%%time
A_mrg = save_minian(A_mrg.rename("A_mrg"), intpath, overwrite=True)
C_mrg = save_minian(C_mrg.rename("C_mrg"), intpath, overwrite=True)
C_chk_mrg = save_minian(
    C_mrg.rename("C_mrg_chk"),
    intpath,
    overwrite=True,
    chunks={"unit_id": -1, "frame": chk["frame"]},
)
sig_mrg = save_minian(sig_mrg.rename("sig_mrg"), intpath, overwrite=True)

CPU times: total: 1.77 s
Wall time: 9.25 s


### Second spatial and temporal updates

#### Spatial update
Generally not much happens at that stage.

In [79]:
%%time
A_secS, mask_secS, norm_fac_secS = update_spatial(
    Y_hw_chk, A_mrg, C_mrg, sn_spatial, **param_second_spatial
)
C_secS = save_minian(
    (C_mrg.sel(unit_id=mask_secS) * norm_fac_secS).rename("C_new"), intpath, overwrite=True
)
C_chk_secS = save_minian(
    (C_chk_mrg.sel(unit_id=mask_secS) * norm_fac_secS).rename("C_chk_new"), intpath, overwrite=True
)

estimating penalty parameter
computing subsetting matrix
fitting spatial matrix


c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 23.89 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 26.60 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


14 out of 415 units dropped
CPU times: total: 2.03 s
Wall time: 3min 29s


#### Update background

In [80]:
%%time
b_secS, f_secS = update_background(Y_fm_chk, A_secS, C_chk_secS)

c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 13.58 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: total: 7.88 s
Wall time: 3min 27s


#### Save spatial update

In [81]:
%%time
A_secS = save_minian(
    A_secS.rename("A"),
    intpath,
    overwrite=True,
    chunks={"unit_id": 1, "height": -1, "width": -1},
)
b_secS = save_minian(b_secS.rename("b"), intpath, overwrite=True)
f_secS = save_minian(
    f_secS.chunk({"frame": chk["frame"]}).rename("f"), intpath, overwrite=True
)
C_secS = save_minian(C_secS.rename("C"), intpath, overwrite=True)
C_chk_secS = save_minian(C_chk_secS.rename("C_chk"), intpath, overwrite=True)

CPU times: total: 2.39 s
Wall time: 18.1 s


#### Second temporal update

In [82]:
%%time
YrA_secT = save_minian(
    compute_trace(Y_fm_chk, A_secS, b_secS, C_chk_secS, f_secS).rename("YrA"),
    intpath,
    overwrite=True,
    chunks={"unit_id": 1, "frame": -1},
)

c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 13.56 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 13.56 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 24.62 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: total: 1.91 s
Wall time: 26 s


In [83]:
%%time
C_secT, S_secT, b0_secT, c0_secT, g_secT, mask_secT = update_temporal(
    A_secS, C_secS, YrA=YrA_secT, **param_second_temporal
)

grouping overlaping units
updating temporal components
38 out of 401 units dropped
CPU times: total: 5.12 s
Wall time: 54.8 s


### Save all

In [84]:
%%time
C_secT = save_minian(
    C_secT.rename("C").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
C_chk_secT = save_minian(
    C_secT.rename("C_chk"),
    intpath,
    overwrite=True,
    chunks={"unit_id": -1, "frame": chk["frame"]},
)
S_secT = save_minian(
    S_secT.rename("S").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
b0_secT = save_minian(
    b0_secT.rename("b0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
c0_secT = save_minian(
    c0_secT.rename("c0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
A_secT = A_secS.sel(unit_id=C_secT.coords["unit_id"].values)

CPU times: total: 828 ms
Wall time: 5.28 s


## Generate videos and close 

!!!!! Only if cnmfviewer has been used

In [89]:
%%time
# Generate video
generate_videos(varr_ref.sel(subset), Y_fm_chk, A=A_secT, C=C_chk_secT, vpath=dpath)

"""
# Generate labels
if interactive:
    A = A_secT.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    C = C_secT.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    S = S_secT.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    c0 = c0_secT.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    b0 = b0_secT.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))"""

generating traces
normalizing
writing videos


c:\Users\Manip2\.conda\envs\minian311new2\Lib\site-packages\distributed\client.py:3162: UserWarning: Sending large graph of size 12.91 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
%%time
# Save final
A = save_minian(A_secT.rename("A"), **param_save_minian)
C = save_minian(C_secT.rename("C"), **param_save_minian)
S = save_minian(S_secT.rename("S"), **param_save_minian)
c0 = save_minian(c0_secT.rename("c0"), **param_save_minian)
b0 = save_minian(b0_secT.rename("b0"), **param_save_minian)
b = save_minian(b_init.rename("b"), **param_save_minian)
f = save_minian(f_init.rename("f"), **param_save_minian)

# Close cluster
#client.close()
#cluster.close()

CPU times: user 342 ms, sys: 67.3 ms, total: 409 ms
Wall time: 496 ms
